In [ ]:
get_ipython().system('pip install google-cloud-bigquery')
get_ipython().system('pip install google-cloud-bigquery[pandas]')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
get_ipython().run_line_magic('matplotlib', 'inline')

In [ ]:
import os
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics

CONFIGURE THE BIGQUERY SETTINGS

In [ ]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [ ]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_training.weather_data`
"""

In [ ]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
data = query_job.to_dataframe()
data['date']= pd.to_datetime(data['date'])
data.head()

In [ ]:
X = data.drop("wind_speed",1)   #Feature Matrix
X = data.drop("date",1)
y = data["wind_speed"]          #Target Variable

In [ ]:
plt.figure(figsize=(12,10))
cor = data.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Correlation with output variable
cor_target = abs(cor["potential_water_deficit"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.5]
print(relevant_features)

In [ ]:
data.plot(subplots=True, figsize=(20,24))

In [ ]:
min(data['date']),max(data['date'])

In [ ]:
data.dtypes

In [ ]:
data = data.set_index('date')

In [ ]:
data.index

In [ ]:
data['Year'] = data.index.year
data['Month'] = data.index.month
# Display a random sampling of 5 rows
data.sample(5, random_state=0)

In [ ]:
data.loc['2019-08']

In [ ]:
data.loc['2019-08']

In [ ]:
sns.set(rc={'figure.figsize':(11, 4)})

In [ ]:
data['precipitation_data'].plot(linewidth=0.5);

In [ ]:
cols_plot = ['max_rel_humidity','max_temperature','mean_temperature','min_rel_humidity','min_temperature','potential_water_deficit','precipitation_data','wind_speed']
axes = data[cols_plot].plot(marker='.', alpha=0.5, linestyle='None', figsize=(30, 25), subplots=True)
for ax in axes:
    ax.set_ylabel('Precipitation')

In [ ]:
import matplotlib.dates as mdates
fig, ax = plt.subplots()
ax.plot(data.loc['2019-08':'2019-12', 'precipitation_data'], marker='o', linestyle='-')
ax.set_ylabel('Precipitation')
ax.set_title('Aug 2019-2020 Precipiation Data')

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(11, 10), sharex=True)
for name, ax in zip(['precipitation_data', 'min_temperature', 'max_temperature'], axes):
    sns.boxplot(data=data, x='Month', y=name, ax=ax)
    ax.set_ylabel('precipitation')
    ax.set_title(name)
    # Remove the automatic x-axis label from all but the bottom subplot
    if ax != axes[-1]:
        ax.set_xlabel('')

In [ ]:
sns.boxplot(data=data, x='Month', y='wind_speed');

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(x=data['max_temperature'], lags=50)

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X,y)

In [ ]:
y_pred = pd.Series(model.predict(X), index=X.index)

In [ ]:
ax = y.plot(alpha=0.5)
ax = y_pred.plot(ax=ax, linewidth=3)

In [ ]:
lag_1 = data['precipitation_data'].shift(1)

In [ ]:
data['lag_1']=lag_1

In [ ]:
X = data.drop("wind_speed",1)   #Feature Matrix
X = data.drop("date",1)
y = data["wind_speed"]          #Target Variable
X.head()